In [1]:
with open('input.txt', 'r') as input_file:
    text = input_file.read()

In [14]:
unique_chars = sorted(list(set(text)))
vocab_size = len(unique_chars)
print(''.join(unique_chars))
print(len(unique_chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [3]:
ctoi = { char:ind for ind,char in enumerate(unique_chars)}
itoc = {ind:char for ind,char in enumerate(unique_chars)}

encode = lambda s: [ctoi[z] for z in s]
decode = lambda i: ''.join(itoc[z] for z in i)

print(encode('hello'))
print(decode(encode('hello')))

[46, 43, 50, 50, 53]
hello


In [4]:
import torch

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
data = torch.tensor(encode(text), dtype = torch.long)
data[:100]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

In [6]:
# train and test split. we are considering 90% of the data as our training data.

factor = int(0.9 * len(data))
train = data[:factor]
val = data[factor:]

In [7]:
blk = 8
print(train[:blk],'\n')

def get_targets(mat, tar, block_size):
    for _ in range(block_size):
        context = mat[:_+1]
        target = tar[_]
        print(f'context:{context}',f'target: {target}\n')


tensor([18, 47, 56, 57, 58,  1, 15, 47]) 



In [8]:
torch.manual_seed(1337)
batch_size = 4
blk_size = 8

def get_batch(split):
    data = train if split == 'train' else val
    indices = torch.randint((len(data) - blk_size), (batch_size,))
    x = torch.stack([data[i:i+blk_size] for i in indices])
    y = torch.stack([data[i+1:i+blk_size+1] for i in indices])
    return x, y

x,y = get_batch('train')

x.shape,y.shape,x,y

(torch.Size([4, 8]),
 torch.Size([4, 8]),
 tensor([[24, 43, 58,  5, 57,  1, 46, 43],
         [44, 53, 56,  1, 58, 46, 39, 58],
         [52, 58,  1, 58, 46, 39, 58,  1],
         [25, 17, 27, 10,  0, 21,  1, 54]]),
 tensor([[43, 58,  5, 57,  1, 46, 43, 39],
         [53, 56,  1, 58, 46, 39, 58,  1],
         [58,  1, 58, 46, 39, 58,  1, 46],
         [17, 27, 10,  0, 21,  1, 54, 39]]))

In [9]:
for _ in range(batch_size):
    get_targets(x[_],y[_],len(x[_]))
    

context:tensor([24]) target: 43

context:tensor([24, 43]) target: 58

context:tensor([24, 43, 58]) target: 5

context:tensor([24, 43, 58,  5]) target: 57

context:tensor([24, 43, 58,  5, 57]) target: 1

context:tensor([24, 43, 58,  5, 57,  1]) target: 46

context:tensor([24, 43, 58,  5, 57,  1, 46]) target: 43

context:tensor([24, 43, 58,  5, 57,  1, 46, 43]) target: 39

context:tensor([44]) target: 53

context:tensor([44, 53]) target: 56

context:tensor([44, 53, 56]) target: 1

context:tensor([44, 53, 56,  1]) target: 58

context:tensor([44, 53, 56,  1, 58]) target: 46

context:tensor([44, 53, 56,  1, 58, 46]) target: 39

context:tensor([44, 53, 56,  1, 58, 46, 39]) target: 58

context:tensor([44, 53, 56,  1, 58, 46, 39, 58]) target: 1

context:tensor([52]) target: 58

context:tensor([52, 58]) target: 1

context:tensor([52, 58,  1]) target: 58

context:tensor([52, 58,  1, 58]) target: 46

context:tensor([52, 58,  1, 58, 46]) target: 39

context:tensor([52, 58,  1, 58, 46, 39]) target:

In [10]:
import torch.nn as nn
from torch.nn import functional as f

In [18]:
torch.manual_seed(1337)

class BigramModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.embedding_table = nn.Embedding(vocab_size,vocab_size)
        
    def forward(self, idx, targets):
        logits = self.embedding_table(idx)
        B,T,C = logits.shape
        logits = logits.view(B*T,C)
        targets = targets.view(B*T)
        loss = f.cross_entropy(logits,targets)
        return loss,logits

model = BigramModel(vocab_size)
loss,logits = model(x,y)

loss,logits.shape

        

(tensor(4.8786, grad_fn=<NllLossBackward0>), torch.Size([32, 65]))